# HADOOP-16435: RpcMetrics should not be retained forever

[Link to Jira report.](https://issues.apache.org/jira/browse/HADOOP-16435)

Package created by Max Goetzmann.

## Reserve bare metal and floating IP

In [10]:
import chi
import uuid
from chi.lease import lease_duration, add_node_reservation, create_lease, add_fip_reservation

SITE = 'CHI@NU'
CHARGE_CODE = 'CHI-220969'
PROJECT_ID = uuid.uuid4()
LEASE_NAME = 'bug-repro-HADOOP-16435'

chi.use_site(SITE)
chi.set('project_name', CHARGE_CODE)
chi.set("project_domain_name", "chameleon")

node_type = "compute_skylake"
start_date, end_date = lease_duration(hours=6)

# Build list of reservations (in this case there is only one reservation)
reservations = []
add_node_reservation(reservations, count=1, node_type=node_type)
add_fip_reservation(reservations, count=1)

# Create the lease
lease = create_lease(LEASE_NAME, reservations, start_date=start_date,
                     end_date=end_date)


Now using CHI@NU:
URL: https://sl-ciab.northwestern.edu
Location: Evanston, Illinois, USA
Support contact: help@chameleoncloud.org


BadRequest: Invalid input for field 'identity/token/id': None is not of type 'string'

Failed validating 'type' in schema['properties']['identity']['properties']['token']['properties']['id']:
    {'type': 'string'}

On instance['identity']['token']['id']:
    None (HTTP 400) (Request-ID: req-a3969944-ebf5-451b-b5a2-5e9d80c43b25)

## Launch bare metal

In [ ]:
from chi.lease import get_node_reservation
from chi.server import create_server, associate_floating_ip, wait_for_tcp

SERVER_NAME = LEASE_NAME
reservation_id = get_node_reservation(LEASE_NAME)
server = create_server(SERVER_NAME, 
                       image_name="CC-Ubuntu22.04",
                       reservation_id=reservation_id)

server_id = server.id
ip = associate_floating_ip(server_id)

# Wait for SSH connectivity over port 22
wait_for_tcp(ip, port=22)

## Upload and execute setup script 

In [ ]:
import os
from chi import ssh

def list_files(dir):
    file_list = []
    for root, _, files in os.walk(dir):
        for file in files:
            file_list.append(dir + os.path.relpath(os.path.join(root, file), dir))
    return file_list

bin_files = list_files('./bin/')
patch_files = list_files('./patch/')
new_files = list_files('./new_files/')

with ssh.Remote(ip) as conn:
    # Upload setup files 
    conn.run(f'mkdir {LEASE_NAME}')
    conn.run(f'cd {LEASE_NAME}')
    for file in bin_files + patch_files + new_files:
        conn.put(file)
    
    # Execute setup
    conn.run("bash prepare.sh")


['./bin/setup.sh', './bin/protobuf_install.sh']


## Perform experiment

In [ ]:
with ssh.Remote(ip) as conn:
    # Run tests
    conn.run("bash run_tests.sh")

## Process and output experiment results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

BUGGY_REPO = 'buggy_repo'
PATCHED_REPO = 'patched_repo'

# Read the CSV files
file1 = f'./{BUGGY_REPO}/heap_usage.csv'
file2 = f'./{PATCHED_REPO}/heap_usage.csv'
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Convert bytes to megabytes
df1.iloc[:, 1] = df1.iloc[:, 1] / 1024 / 1024
df2.iloc[:, 1] = df2.iloc[:, 1] / 1024 / 1024

# Plot the graph
plt.figure(figsize=(10, 6))
plt.plot(df1.iloc[:, 0], df1.iloc[:, 1], label='Before Patch')
plt.plot(df2.iloc[:, 0], df2.iloc[:, 1], label='After Patch')
plt.xlabel('Number of Sessions')
plt.ylabel('Heap Space (MB)')
plt.title('Number of Sessions vs Heap Space')
plt.legend()

# Save the plot to a file
plt.savefig('output_plot.png')
plt.show()